In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False

# Create a connection to the database
%sql duckdb:///chinook.sqlite

Connecting to 'duckdb:///chinook.sqlite'

In [31]:
# Find outliers in the data using window functions and IQRs

# We'll use quantile_cont() to calculate the 1st quartile and percentile_cont() for the 3rd quartile
# The functions do the same thing, but the former is ANSI SQL standard while the latter is PostgreSQL specific

# Find the average track length for each genre along with standard deviation
# First using GROUP BY
%sql q_group << \
     SELECT t.genreid, g.name AS genre \
          , AVG(milliseconds) as avg_length \
          , STDDEV(milliseconds) as std_length \
          , quantile_cont(milliseconds, 0.25) AS q1 \
          , percentile_cont(0.75) WITHIN GROUP (ORDER BY milliseconds) as q3 \
       FROM tracks t \
       JOIN genres g \
         ON t.genreid = g.genreid \
      GROUP BY g.name, t.genreid \
      ORDER BY t.genreid ASC

# Now using window functions
%sql q_window << \
     SELECT DISTINCT t.genreid, g.name AS genre \
          , AVG(milliseconds) OVER (PARTITION BY t.genreid) as avg_length \
          , STDDEV(milliseconds) OVER (PARTITION BY t.genreid) as std_length \
          , quantile_cont(milliseconds, 0.25) OVER (PARTITION BY t.genreid) AS q1 \
          , quantile_cont(milliseconds, 0.75) OVER (PARTITION BY t.genreid) AS q3 \
       FROM tracks t \
       JOIN genres g \
         ON t.genreid = g.genreid \
      ORDER BY t.genreid ASC

# Compare the results
display(q_group.head())
display(q_window.head())

,GenreId,genre,avg_length,std_length,q1,q3
0,1,Rock,283910.043177,126795.563866,215510.00,319582.00
1,2,Jazz,291755.376923,129124.061679,202219.75,321299.25
2,3,Metal,309749.443850,107888.758634,245198.25,366543.50
3,4,Alternative & Punk,234353.849398,78719.235098,192711.50,270582.50
4,5,Rock And Roll,134643.500000,20478.478435,114017.75,144770.25


,GenreId,genre,avg_length,std_length,q1,q3
0,1,Rock,283910.043177,126795.563866,215510.00,319582.00
1,2,Jazz,291755.376923,129124.061679,202219.75,321299.25
2,3,Metal,309749.443850,107888.758634,245198.25,366543.50
3,4,Alternative & Punk,234353.849398,78719.235098,192711.50,270582.50
4,5,Rock And Roll,134643.500000,20478.478435,114017.75,144770.25


In [42]:
# Now we'll use the window function to calculate the IQR for each genre and identify outliers
# We'll use the `q_window` dataframe from the previous query

%sql q_outliers << \
     SELECT t.name AS track, w.genre \
          , t.milliseconds \
          , (t.milliseconds - w.q1) / (w.q3 - w.q1) AS iqr_distance \
          , (t.milliseconds - w.avg_length) / w.std_length AS z_score \
       FROM tracks t \
       JOIN q_window w \
         ON t.genreid = w.genreid \
      WHERE abs(iqr_distance) > 1.5 AND abs(z_score) > 2 \
      ORDER BY abs(iqr_distance) desc

# Display the outliers
display(q_outliers.head())

,track,genre,Milliseconds,iqr_distance,z_score
0,Through a Looking Glass,Drama,5088838,76.610428,4.856160
1,LOST Season 4 Trailer,Drama,112712,-76.396390,-4.757663
2,LOST In 8:15,Drama,497163,-64.575222,-4.014907
3,"Crossroads, Pt. 1",Sci Fi & Fantasy,2622622,-43.492391,-4.600956
4,Hero,Science Fiction,2713755,33.908152,2.823665
